In [1]:
!pwd

/home/ec2-user/SageMaker/oss-llm-inference/summarize-legal-text


In [1]:
# !pip install -r requirements.txt

In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM
from datasets import load_dataset

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [3]:
model_id = "anurag-208/summarize_legal_text"
base_model_id = "google/flan-t5-large"

config = PeftConfig.from_pretrained(model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)

# Merging: Add adapter weights to the original model
model = PeftModel.from_pretrained(base_model,
                                  model_id)

In [4]:
model.eval()  # evaluation mode i.e. no training

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict

In [5]:
dataset = load_dataset("billsum")

dataset

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

Example 1

In [6]:
sample = dataset['test'][123]

sample['text']

"SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Venezuelan Refugee Assistance Act''.\n\nSEC. 2. ADJUSTMENT OF STATUS OF CERTAIN VENEZUELAN NATIONALS.\n\n    (a) Adjustment of Status.--\n            (1) In general.--Notwithstanding section 245(c) of the \n        Immigration and Nationality Act (8 U.S.C. 1255(c)), the status \n        of any alien described in subsection (b) shall be adjusted by \n        the Secretary of Homeland Security to that of an alien lawfully \n        admitted for permanent residence, if the alien--\n                    (A) applies for such adjustment before January 1, \n                2019;\n                    (B) is not inadmissible under paragraph (1), (2), \n                (3), (4), (6)(E), (6)(G), (8), (10)(A), (10)(C), or \n                (10)(D) of section 212(a) of the Immigration and \n                Nationality Act (8 U.S.C. 1182(a));\n                    (C) is not deportable under paragraph (1)(E), \n                (1)(G), (2),

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
input_ids = tokenizer(sample["text"],
                      return_tensors="pt",
                      truncation=True).input_ids

In [9]:
type(input_ids)

torch.Tensor

In [10]:
outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=64,
                         do_sample=True,
                         top_p=0.8)

In [11]:
print(outputs)

tensor([[    0,   304,   370,    21,     8, 13592,    13,  2637,    13,   824,
         17841,    29,  1157,     7,     5,     1]])


In [12]:
tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                       skip_special_tokens=True)[0]

'To provide for the adjustment of status of certain Venezuelan nationals.'

Example 2

In [13]:
sample = dataset['test'][234]

sample['text']

"SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Education for Democracy Act''.\n\nSEC. 2. CIVIC EDUCATION.\n\n    Part F of title X of the Elementary and Secondary Education Act of \n1965 (20 U.S.C. 8141 et seq.) is amended to read as follows:\n\n                       ``PART F--CIVIC EDUCATION\n\n``SEC. 10601. SHORT TITLE.\n\n    ``This part may be cited as the `Education for Democracy Act'.\n\n``SEC. 10602. FINDINGS.\n\n    ``The Congress finds that--\n            ``(1) college freshmen surveyed in 1997 by the Higher \n        Education Research Institute at the University of California at \n        Los Angeles demonstrated higher levels of disengagement, both \n        academically and politically, than any previous entering class \n        of students;\n            ``(2) college freshmen in 1997 demonstrated the lowest \n        levels of political interest in the 20-year history of surveys \n        conducted by the Higher Education Research Institute at the \n     

In [14]:
input_ids = tokenizer(sample["text"],
                      return_tensors="pt",
                      truncation=True).input_ids

outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=64,
                         do_sample=True,
                         top_p=0.8)

tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                       skip_special_tokens=True)[0]

'Part F of title X of the Elementary and Secondary Education Act of 1965 (20 U.S.C. 8141 et seq.) is amended to read as follows: PART F--CIVIC EDUCATION SEC. 10601. SHORT TIT'

Example 3

In [60]:
sample = dataset['test'][400]

sample['text']

"SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Abraham Lincoln Bicentennial 1-Cent \nCoin Redesign Act''.\n\nSEC. 2. FINDINGS.\n\n    The Congress finds as follows:\n            (1) Abraham Lincoln, the 16th President, was one of the \n        Nation's greatest leaders, demonstrating true courage during \n        the Civil War, one of the greatest crises in the Nation's \n        history.\n            (2) Born of humble roots in Hardin County, Kentucky, on \n        February 12, 1809, Abraham Lincoln rose to the Presidency \n        through a combination of honesty, integrity, intelligence, and \n        commitment to the United States.\n            (3) With the belief that all men are created equal, Abraham \n        Lincoln led the effort to free all slaves in the United States.\n            (4) Abraham Lincoln had a generous heart, with malice \n        toward none and with charity for all.\n            (5) Abraham Lincoln gave the ultimate sacrifice for the \n      

In [61]:
input_ids = tokenizer(sample["text"],
                      return_tensors="pt",
                      truncation=True).input_ids

outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=64,
                         do_sample=True,
                         top_p=0.8)

tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                       skip_special_tokens=True)[0]

"To require the redesign of the reverse of the Lincoln cent'' coin, and for other purposes."

Example 4

In [20]:
sample = dataset['test'][567]

sample['text']

"SECTION 1. SHORT TITLE; TABLE OF CONTENTS.\n\n    (a) Short Title.--This Act may be cited as the ``Secondary Metal \nTheft Prevention Act of 2009''.\n    (b) Table of Contents.--The table of contents for this Act is as \nfollows:\n\nSec. 1. Short title; table of contents.\nSec. 2. Findings.\nSec. 3. Sense of the Congress.\nSec. 4. Purposes.\nSec. 5. Purchase transaction requirements.\nSec. 6. Protection of personal information.\nSec. 7. Enforcement by Federal Trade Commission.\nSec. 8. Enforcement by State Attorneys General.\nSec. 9. Civil penalty.\nSec. 10. State and local law not preempted.\nSec. 11. Definitions.\nSec. 12. Effective date.\n\nSEC. 2. FINDINGS.\n\n    The Congress finds the following:\n            (1) Since 2006, metal theft has been on the rise, largely \n        due to a surge in the global demand for scrap metal and a \n        historic increase in the price of metal commodities.\n            (2) Theft of metal components is jeopardizing the integrity \n        of 

In [21]:
input_ids = tokenizer(sample["text"],
                      return_tensors="pt",
                      truncation=True).input_ids

outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=64,
                         do_sample=True,
                         top_p=0.8)

tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                       skip_special_tokens=True)[0]

'To protect the public from theft of metals.'

Example 5

In [26]:
sample = dataset['test'][135]

sample['text']

"SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Civil Rights Act of 1964 \nCommemorative Coin Act''.\n\nSEC. 2. FINDINGS.\n\n    Congress finds that--\n            (1) on December 1, 1955, Rosa Parks' brave act of defiance, \n        refusing to give up her seat to a white person on a segregated \n        bus in Montgomery, Alabama, galvanized the modern civil rights \n        movement and led to the desegregation of the South;\n            (2) on February 1, 1960, 4 college students, Joseph McNeil, \n        Franklin McCain, David Richmond, and Ezell Blair, Jr., asked to \n        be served at a lunch counter in Greensboro, North Carolina, and \n        lunch counter sit-ins began to occur throughout the South to \n        challenge segregation in places of public accommodation;\n            (3) on May 4, 1961, the Freedom Rides into the South began \n        to test new court orders barring segregation in interstate \n        transportation, and riders were jailed and 

In [27]:
input_ids = tokenizer(sample["text"],
                      return_tensors="pt",
                      truncation=True).input_ids

outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=64,
                         do_sample=True,
                         top_p=0.8)

tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                       skip_special_tokens=True)[0]

'To commemorate the tenth anniversary of the passage of the Civil Rights Act of 1964 by commemorating coins in commemoration of Rosa Parks, who, on December 1, 1955, refused to give up her seat to a white person on a segregated bus in Montgomery, Alabama, galvanized'